In [1]:
import re #https://www.letscode.com.br/blog/expressoes-regulares-parte-i
import pypyodbc # para conectar a um Access DB
import requests # para baixar um arquivo usando python
import zipfile # para trabalhar com zipfiles
from zipfile import ZipFile
import pandas as pd
import PyPDF2 # Importing required modules for dealing with PDFs

In [2]:
# Creating a pdf file object
pdfFileObj = open('PR-LDW - ADs.pdf','rb')
# Creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# Getting number of pages in pdf file
pages = pdfReader.numPages
# Loop for reading all the Pages
ADsALL = [] # cria listas vazias, para colocar todas as ADs conforme o padrão definido
ADsFAAALL = []
ADsEASAALL = []

In [4]:
# aqui tem os vários padrões de escrita de ADs
padraoFAA = r'AD \d{4}-\d{2}-\d{2}'
padraoFAA2 = r'AD \d{2}-\d{2}-\d{2}'
padraoFAALider1 = r'\d{4}-\d{2}-\d{2}'
padraoFAALider2 = r'\d{2}-\d{2}-\d{2}'
padraoEASA = r'AD \d{4}-\d{4}'
padraoEASALider1 = r'20\d{2}-\d{4}'
padraoEASALider2 = r'19\d{2}-\d{4}'
for i in range(pages):
        # Creating a page object
        pageObj = pdfReader.getPage(i)
        # Printing Page Number
        #print("Page No: ",i)
        # Extracting text from page
        # And splitting it into chunks of lines
        text = pageObj.extractText().split("   ")
        # using list comprehension
        PdfTextasString = ' '.join([str(elem) for elem in text]) # convert text to string
        #https://www.geeksforgeeks.org/python-program-to-convert-a-list-to-string/
        ADsFAA = re.findall(padraoFAA, PdfTextasString) 
        ADsFAA2 = re.findall(padraoFAA2, PdfTextasString)        
        ADsFAALider1 = re.findall(padraoFAALider1, PdfTextasString) 
        ADsFAALider2 = re.findall(padraoFAALider2, PdfTextasString) 
        ADsFAAALL = ADsFAAALL+ADsFAA+ADsFAA2+ADsFAALider1+ADsFAALider2 # lista todos os caracteres que parecem ADs da FAA
        ADsEASA = re.findall(padraoEASA, PdfTextasString) 
        ADsEASALider1 = re.findall(padraoEASALider1, PdfTextasString) 
        ADsEASALider2 = re.findall(padraoEASALider2, PdfTextasString) 
        ADsEASAALL = ADsEASAALL+ADsEASA+ADsEASALider1+ADsEASALider2# lista todos os caracteres que parecem ADs da EASA
        ADsALL = ADsALL+ADsFAALider1+ADsFAALider2+ADsEASALider1+ADsEASALider2
        # Finally the lines are stored into list
        # For iterating over list a loop is used
        #for j in range(len(text)):
                # Printing the line
                # Lines are separated using "\n"
              #  print(text[j])#,end="\n\n")
        # For Separating the Pages
       # print()
# closing the pdf file object
pdfFileObj.close()

In [5]:
ADsFAAALL= pd.DataFrame(ADsFAAALL,columns=['ADs'])
ADsFAAALL = pd.DataFrame(ADsFAAALL.ADs.unique(),columns=['ADs'])
ADsFAAALL['Orig'] = 'FAA' #classifico as ADs do PDF como FAA
ADsEASAALL= pd.DataFrame(ADsEASAALL,columns=['ADs'])
ADsEASAALL = pd.DataFrame(ADsEASAALL.ADs.unique(),columns=['ADs'])
ADsEASAALL['Orig'] = 'EASA' #classifico as ADs do PDF como EASA
dfADsALL = pd.concat([ADsFAAALL,ADsEASAALL]) # junto tudo
dfADsALL.ADs = dfADsALL.ADs.str.lstrip('AD ') # tira os começos "AD"

In [8]:
dfSBsALL = dfADsALL[dfADsALL['ADs'].astype(str).str.startswith('76')] # todos os ASB achados
dfADsALL = dfADsALL[~dfADsALL['ADs'].astype(str).str.startswith('76')] # somente as que parecem ADs

In [9]:
# Este comando extrai da FAA todas as ADs emitidas e salva o zip na pasta.
# https://rgl.faa.gov/Regulatory_and_Guidance_Library/rgAD.nsf/MainFrame?OpenFrameSet
# https://rgl.faa.gov/Regulatory_and_Guidance_Library/rgad.nsf/1f3d470805e57a0786257bce004f3976/d7876e8a7bb0e2aa86257beb0051e285/$FILE/AD.zip
url = 'https://rgl.faa.gov/Regulatory_and_Guidance_Library/rgad.nsf/1f3d470805e57a0786257bce004f3976/d7876e8a7bb0e2aa86257beb0051e285/$FILE/AD.zip'
r = requests.get(url, allow_redirects=True)
# carrega o arquivo que se deseja baixar
open('ADES.zip', 'wb').write(r.content) #pode-se substituir o ADES.zip pela url
#salva com o nome que eu quero. Leva uns 20 segundos e cospe o tamanho do arquivo em bytes
#"""Extract example .zip file into this directory"""
archive = zipfile.ZipFile('ADES.zip', 'r') # extrai o zip e temos o ACCDB file
archive.extractall(path=None, members=None, pwd=None)

In [10]:
# estes comandos trabalham o Access CB
pypyodbc.lowercase = False
PAFF = r''
conn = pypyodbc.connect(
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};" +
    r"Dbq=e:\OneDrive\Documentos\LCN Consultoria\Tests on BD\AD.accdb;")
cur = conn.cursor()
query = 'SELECT * FROM ADs_by_Model' # monta uma "query" com as ADs por modelo do Access DB
dataf = pd.read_sql(query, conn)

In [11]:
# Aqui eu escolho o modelo do qual quero checar as ADs emitidas na FAA
select_model = 'S-76C' # S-76C, S-76A, AW139, 
ADs_Issued_FAA = dataf[dataf.Model==select_model]

In [14]:
ADs_Issued_FAA

,Make,Model,ADNumber
125571,Sikorsky Aircraft Corporation,S-76C,2000-11-52
125572,Sikorsky Aircraft Corporation,S-76C,2000-23-51
125573,Sikorsky Aircraft Corporation,S-76C,2000-23-52
125574,Sikorsky Aircraft Corporation,S-76C,2001-01-04
125575,Sikorsky Aircraft Corporation,S-76C,2001-03-51
125576,Sikorsky Aircraft Corporation,S-76C,2001-10-06
125577,Sikorsky Aircraft Corporation,S-76C,2001-25-06
125578,Sikorsky Aircraft Corporation,S-76C,2002-15-51
125579,Sikorsky Aircraft Corporation,S-76C,2002-21-07
125580,Sikorsky Aircraft Corporation,S-76C,2003-04-15


In [13]:
df_FAA_Ads = dfADsALL[dfADsALL.Orig=='FAA']
df_EASA_Ads = dfADsALL[dfADsALL.Orig=='EASA']

In [24]:
df_FAA_Ads.dtypes

ADs     object
Orig    object
dtype: object

In [ ]:
# só consegui trazer as ADs do site colando as tabelas no excel e depois colando só a coluna que interessa num txt
# e salvando como csv
ADsfromsite = pd.read_csv('ADs.csv')

In [15]:
# Controlados mas não listados como mandatórios para aeronave/motor
Controlled_Not_Listed_FAA = df_FAA_Ads.join(ADs_Issued_FAA, lsuffix="ADs", rsuffix="ADNumber")
Controlled_Not_Listed_FAA = Controlled_Not_Listed_FAA[Controlled_Not_Listed_FAA.ADNumber.isna()]
# listados como mandatórios para aeronave/motor mas não Controlados
Listed_Not_Controlled_FAA = ADs_Issued_FAA.join(df_FAA_Ads, lsuffix='ADNumber', rsuffix='ADs')
Listed_Not_Controlled_FAA = Listed_Not_Controlled_FAA[Listed_Not_Controlled_FAA.ADs.isna()]

In [25]:
Controlled_Not_Listed_FAA

,ADs,Orig,Make,Model,ADNumber
0,2000-11-52,FAA,NaN,NaN,NaN
1,2001-01-04,FAA,NaN,NaN,NaN
2,2001-03-51,FAA,NaN,NaN,NaN
3,2001-10-06,FAA,NaN,NaN,NaN
4,2001-22-14,FAA,NaN,NaN,NaN
...,...,...,...,...,...
123,84-10-02,FAA,NaN,NaN,NaN
125,2071-31-10,FAA,NaN,NaN,NaN
126,71-31-10,FAA,NaN,NaN,NaN
127,2002-05-06,FAA,NaN,NaN,NaN


In [26]:
#Esta é a lista de ADs em dia sem menção no controle do PDF
Listed_Not_Controlled_FAA

,Make,Model,ADNumber,ADs,Orig
125571,Sikorsky Aircraft Corporation,S-76C,2000-11-52,NaN,NaN
125572,Sikorsky Aircraft Corporation,S-76C,2000-23-51,NaN,NaN
125573,Sikorsky Aircraft Corporation,S-76C,2000-23-52,NaN,NaN
125574,Sikorsky Aircraft Corporation,S-76C,2001-01-04,NaN,NaN
125575,Sikorsky Aircraft Corporation,S-76C,2001-03-51,NaN,NaN
125576,Sikorsky Aircraft Corporation,S-76C,2001-10-06,NaN,NaN
125577,Sikorsky Aircraft Corporation,S-76C,2001-25-06,NaN,NaN
125578,Sikorsky Aircraft Corporation,S-76C,2002-15-51,NaN,NaN
125579,Sikorsky Aircraft Corporation,S-76C,2002-21-07,NaN,NaN
125580,Sikorsky Aircraft Corporation,S-76C,2003-04-15,NaN,NaN


In [ ]:
Controled_Not_Listed[Controled_Not_Listed.AD_Number.isna()]

In [ ]:
# Até aqui, esse python serviu para extrair todos os códigos que parecem com ADs e
# comparar com uma lista de ADs emitidas (Listed)
# os próximos passos seriam conseguir extrair diretamente da FAA ou da EASA a lista de ADs, 
# Também seria muito bom conseguir reconverter os PDF de apresentação de controle em tabelas completas, 
# com todo o controle e informações sobre cada AD

In [ ]:
STC_FAA = pd.read_csv('STC.zip',encoding='cp1252') #https://rgl.faa.gov/Regulatory_and_Guidance_Library/rgstc.nsf/6e2e3b6e3510840b86257bcf00638d4a/32e8d50b3ded4bfe86257beb0053a7a0/$FILE/STC.zip
# https://rgl.faa.gov/Regulatory_and_Guidance_Library/rgSTC.nsf/MainFrame?OpenFrameSet
# este CSV tem todos os dados de STC emitidos pela FAA, atualizados mensalmente

In [ ]:
#PAFF = '{0}'.format(os.getcwd())
#PAFF = PAFF+'\\AD.accdb'
#PAFF = print(PAFF)
#PAFF --> aqui a ideia era poder automatizar o nome do arquivo.
# descobri que o connect só funciona com o nome absoluto do caminho do accdb, não com o nome do accdb no working folder

In [ ]:
query2 = 'SELECT mk.Make, m.Model, a.ADNumber, a.SupersededBy FROM ((ADs AS a INNER JOIN Models_to_ADs AS mta ON a.AD_ID = mta.AD_ID) INNER JOIN Models AS m ON mta.Model_ID = m.Model_ID) INNER JOIN Makes AS mk ON m.Make_ID = mk.Make_ID'
dataf2 = pd.read_sql(query2, conn)

In [ ]:
dataf2